# 地图与地理坐标可视化

* geo 

* map

* Bmap

In [1]:
#如果是pyecharts更早的版本可能不需要运行下面的命令
import pyecharts
pyecharts.globals._WarningControl.ShowWarning = False

import warnings
warnings.filterwarnings('ignore')

#隐藏警告

In [2]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import pyecharts as pe
from collections import Counter

## geo

* 数据形式

* 常用设置：去标识，颜色控制

In [3]:
# 官网的案例
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType


g0 = Geo()
g0.add_schema(maptype="china")
g0.add("geo", [list(z) for z in zip(Faker.provinces, Faker.values())])
g0.set_series_opts(label_opts=opts.LabelOpts(is_show=False))#去掉标识
g0.set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True),#显示坐下角的颜色控制
            title_opts=opts.TitleOpts(title="Geo-基本示例"),
        )
g0.render_notebook()

In [4]:
# 地图可视化的数据结构
[list(z) for z in zip(Faker.provinces, Faker.values())]
#注意在pyecharts的后台，中国各个省市的经纬度都是准备好了，不用指定

[['广东', 41],
 ['北京', 24],
 ['上海', 119],
 ['江西', 145],
 ['湖南', 148],
 ['浙江', 121],
 ['江苏', 40]]

In [32]:
#help(g0.add_schema)

* 可以通过maptype来选择具体省市的地图

In [5]:
g0 = Geo()
g0.add_schema(maptype="浙江")#可以通过maptype来选择具体省市的地图
g0.add("geo", [["杭州",50],["宁波",30],["义乌",20],["萧山",99]])
g0.set_series_opts(label_opts=opts.LabelOpts(is_show=False))#去掉标识
g0.set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=300),
            title_opts=opts.TitleOpts(title="Geo-基本示例"),
        )
g0.render_notebook()

## 利用g.add_coordinate画自定义坐标

经纬度查询：https://jingweidu.51240.com/

下面案例参考自：https://blog.csdn.net/qq_31362537/article/details/90667814

颜色码表：https://www.114la.com/other/rgb.htm

In [8]:
from pyecharts.globals import GeoType

In [24]:
city = '杭州'
g = Geo()
g.add_schema(maptype=city)

g.add_coordinate('杭州师范大学仓前校区',120.0109472358, 30.2910373720)#注意这里要求经度在前维度在后
g.add_coordinate('萧山城厢街道',120.2740081170, 30.1605364542)

data_pair = [('杭州师范大学仓前校区', 100),('萧山城厢街道', 500),("杭州",50)]


g.add('', data_pair,symbol_size=20)

#  is_piecewise 是否自定义分段， 变为true 才能生效
g.set_global_opts(
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=800),#注意默认max_
        title_opts=opts.TitleOpts(title="{}-店铺分布".format(city)),
    )

g.render_notebook()

## 标记类型

* from pyecharts.globals import GeoType

* Geo 图类型，有 scatter, effectScatter, heatmap, lines 4 种，建议使用  GeoType.GeoType.EFFECT_SCATTER，GeoType.HEATMAP，GeoType.LINES 将数据添加到地图上

* scatter, effectScatter, heatmap 对应于同一种数据类型；lines则需要另外一种

In [45]:
g0 = Geo()
g0.add_schema(maptype="浙江")#可以通过maptype来选择具体省市的地图
g0.add("geo", [["杭州",50],["宁波",30],["义乌",20],["萧山",99]],type_=GeoType.EFFECT_SCATTER)
g0.set_series_opts(label_opts=opts.LabelOpts(is_show=False))#去掉标识
g0.set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=300),
            title_opts=opts.TitleOpts(title="Geo-基本示例"),
        )
g0.render_notebook()

In [48]:
#注意数据类型的变化
g0 = Geo()
g0.add_schema(maptype="浙江")#可以通过maptype来选择具体省市的地图
g0.add("geo", [["杭州","宁波"],["宁波","义乌"],["义乌","杭州"],["萧山","义乌"]],type_=GeoType.LINES)
g0.set_series_opts(label_opts=opts.LabelOpts(is_show=False))#去掉标识
g0.render_notebook()

## 更多设置案例

* pieces

In [11]:
city = '杭州'
g = Geo()
g.add_schema(maptype=city)
# 定义坐标对应的名称，添加到坐标库中 add_coordinate(name, lng, lat)
g.add_coordinate('杭州师范大学仓前校区',120.0109472358, 30.2910373720)#注意这里要求经度在前维度在后
g.add_coordinate('萧山城厢街道',120.2740081170, 30.1605364542)
# 定义数据对
data_pair = [('杭州师范大学仓前校区', 10),('萧山城厢街道', 5),("杭州",50)]


g.add('', data_pair, type_=GeoType.EFFECT_SCATTER, symbol_size=15)
# 设置样式
g.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

# 自定义分段 color 可以用取色器取色
pieces = [
        {'max': 6, 'label': '5以下', 'color': '#FF00FF'},
        {'min': 6, 'max': 10, 'label': '5-10', 'color': 'yellow'},
        {'min': 10, 'max': 100, 'label': '10-100', 'color': 'red'} # 有下限无上限
    ]
#  is_piecewise 是否自定义分段， 变为true 才能生效
g.set_global_opts(
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces),#注意默认max_
        title_opts=opts.TitleOpts(title="{}-店铺分布".format(city)),
    )

g.render_notebook()

In [35]:
[data[0][0:2]]

[[120.13018670197, 30.238217082645], [120.12937154076, 30.237475794242]]

In [37]:
city = '杭州'
g = Geo()
g.add_schema(maptype=city)
g.add_coordinate('杭州师范大学仓前校区',120.0109472358, 30.2910373720)#注意这里要求经度在前维度在后
g.add_coordinate('萧山城厢街道',120.2740081170, 30.1605364542)
data_pair = [('杭州师范大学仓前校区', 10),('萧山城厢街道', 5),("杭州",50)]

# GeoType.GeoType.EFFECT_SCATTER，GeoType.HEATMAP，GeoType.LINES

g.add('', [['杭州师范大学仓前校区','萧山城厢街道']], type_=GeoType.LINES, symbol_size=15)
# 设置样式
g.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

g.render_notebook()

## 课间题

* 1，引入shoes数据，根据如下步骤作图

In [49]:
a=pd.read_csv(r"D:\try\shoes.csv")

In [53]:
a.head(2)

,_id.$oid,info.上市年份季节,info.上市时间,info.产品名称,info.低帮鞋品名,info.功能,info.吊牌价,info.品牌,info.图案,info.场合,...,info.鞋面材质,info.颜色分类,info.风格,itemid,location,nick,price,sales,title,url
0,5aa77041be9b0338dc1faab4,2018年春季,NaN,NaN,商务休闲鞋,轻质,NaN,YEARCON/意尔康,纯色,日常,...,头层牛皮（除牛反绒）,黑色土黄,商务,562121114807,浙江 丽水,意尔康皮鞋旗舰店,269.0,1583人付款,意尔康男鞋2018春季新款英伦真皮商务休闲皮鞋青年男士皮鞋子男潮,http://detail.tmall.com/item.htm?id=5621211148...
1,5aa77042be9b0338dc1faab5,2017年秋季,NaN,NaN,商务休闲鞋,轻质,NaN,YEARCON/意尔康,纯色,办公室,...,头层牛皮（除牛反绒）,黑色棕色黑色加绒版棕色加绒版黑色镂空棕色镂空,商务,545941839337,浙江 丽水,意尔康皮鞋旗舰店,199.0,1607人付款,意尔康旗舰店官方店男鞋春秋真皮爸爸鞋中老年人商务休闲皮鞋子男,http://detail.tmall.com/item.htm?id=5459418393...


In [20]:
#a.location.str.split(" ",expand=True)[0]

In [51]:
sheng=a.location.str.split(" ",expand=True)[0]

In [52]:
# 提取出location中省与直辖市
sheng.head()

0    浙江
1    浙江
2    浙江
3    浙江
4    浙江
Name: 0, dtype: object

In [54]:
# 以省与直辖市序列做groupby，求出各个省与直辖市的商品数量


In [55]:
p0

0
上海     154
北京     339
吉林       1
四川       4
安徽      24
山东      44
山西      39
广东     284
江苏     120
江西      42
河北      86
河南     393
浙江    2470
湖北      27
福建     952
贵州       2
辽宁       3
重庆       3
dtype: int64

In [56]:
# 把数据处理成地图需要的格式


[('上海', 154),
 ('北京', 339),
 ('吉林', 1),
 ('四川', 4),
 ('安徽', 24),
 ('山东', 44),
 ('山西', 39),
 ('广东', 284),
 ('江苏', 120),
 ('江西', 42),
 ('河北', 86),
 ('河南', 393),
 ('浙江', 2470),
 ('湖北', 27),
 ('福建', 952),
 ('贵州', 2),
 ('辽宁', 3),
 ('重庆', 3)]

In [57]:
# 画出商品数量全国分布图


* 2 画出你家乡城市的地图，通过自定义坐标点，在家乡城市地图上画出你的家，小学，中学的位置

* 3 按照如下步骤画出浙江省各个地市的商品分布图

In [27]:
#a.location.str.split(" ")

In [58]:
#把数据中所有浙江省的商品都挑出来，并以浙江的地市作为值形成一个新的序列
k=0
zj={}
for i in a.location.str.split(" ").items():
    if i[1][0]=="浙江":
        zj.update({i[0]:i[1][1]})
zj=pd.Series(zj)
zj

0       丽水
1       丽水
2       温州
3       温州
4       丽水
        ..
4958    温州
4960    温州
4961    温州
4964    温州
4965    温州
Length: 2470, dtype: object

In [59]:
#求出浙江省各个地市的商品数


丽水     938
台州       2
嘉兴       4
宁波       8
杭州     107
温岭      10
温州    1275
湖州     118
绍兴       1
衢州       3
金华       4
dtype: int64

## map作图

* map的功能和geo有不少重合，区别在于可以显示地图区域块

In [61]:
from pyecharts.charts import Map

In [62]:
[list(z) for z in zip(Faker.provinces, Faker.values())]

[['广东', 67],
 ['北京', 116],
 ['上海', 39],
 ['江西', 119],
 ['湖南', 32],
 ['浙江', 54],
 ['江苏', 56]]

In [63]:
m1=Map()
m1.add("商家A", [list(z) for z in zip(Faker.provinces, Faker.values())], "china")
m1.set_global_opts(
            title_opts=opts.TitleOpts(title="Map-VisualMap（分段型）"),
            visualmap_opts=opts.VisualMapOpts(max_=200, is_piecewise=True),
        )
m1.render_notebook()

## 课间题

* 用map做出男鞋商品全国省分布图

In [64]:
#m1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))


* 做出浙江省地级市商品分布图(注意这里的城市名后面要加"市"，比如丽水市)

In [70]:
data=[]
for i in p1.items():
    data.append([i[0]+"市",i[1]])#注意这里城市的名称后面要加“市“

In [71]:
data

[['丽水市', 938],
 ['台州市', 2],
 ['嘉兴市', 4],
 ['宁波市', 8],
 ['杭州市', 107],
 ['温岭市', 10],
 ['温州市', 1275],
 ['湖州市', 118],
 ['绍兴市', 1],
 ['衢州市', 3],
 ['金华市', 4]]

## 基于百度api的地理可视化

* 申请百度api的方法 见老师从前视频6分10秒处：

https://www.bilibili.com/video/BV1E7411w7Zq?p=17


* 如果用老师的api，请勿滥用（短时间内多次频繁调用）导致此key被封,进而影响其他同学的学习



In [4]:
from pyecharts.globals import ChartType, SymbolType
from pyecharts.charts import BMap

In [5]:
BAIDU_AK = "K2n5m9GXasQFV5arkjpCGsXL"
b1=BMap()
b1.add_schema(
            baidu_ak=BAIDU_AK,
            center=[120.13066322374, 30.240018034923],#这里的地图选择是通过中心点来实现的
        )
b1.add(
            "bmap",
            [list(z) for z in zip(Faker.provinces, Faker.values())],
            label_opts=opts.LabelOpts(formatter="{b}"),
        )
b1.set_global_opts(title_opts=opts.TitleOpts(title="BMap-基本示例"))
b1.render_notebook()

* 使用bmap做商品全国分布图

In [6]:

BAIDU_AK = "K2n5m9GXasQFV5arkjpCGsXL"
b1=BMap()
b1.add_schema(
            baidu_ak=BAIDU_AK,
            center=[120.13066322374, 30.240018034923],
        )
b1.add(
            "bmap",
            [z for z in p0.items()],
            label_opts=opts.LabelOpts(formatter="{b}"),
        )
b1.set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=3000),
            title_opts=opts.TitleOpts(title="BMap-基本示例"),
        )
b1.render_notebook()

NameError: name 'p0' is not defined

* 使用bmap做出浙江省的地市商品分布图

* 注意调节中心点，和zoom

* 注意map_style

清新蓝风格(light):{style:'light'}{style:'dark'}{style:‘redalert’}精简风格(googlelite):{style:'googlelite'}....

https://blog.csdn.net/weixin_41290949/article/details/108488698

In [7]:
BAIDU_AK = "K2n5m9GXasQFV5arkjpCGsXL"
b1=BMap()
b1.add_schema(
            baidu_ak=BAIDU_AK,
            center=[120.13066322374, 30.240018034923],zoom=8,map_style={"style":'hardedge'}
        )
b1.add(
            "bmap",
            [z for z in p1.items()],
            label_opts=opts.LabelOpts(formatter="{b}"),
        )
b1.set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=2000,pos_bottom=50),
            title_opts=opts.TitleOpts(title="BMap-基本示例"),
        )
b1.render_notebook()

NameError: name 'p1' is not defined

## 基于Bmap线图

来自官网案例：https://gallery.pyecharts.org/#/BMap/hiking_trail_in_hangzhou

* 不同于map与geo，type_=ChartType.LINES时，Bmap能直接识别坐标，注意此时的数据结构



* 注意map_style的强大修饰能力，map_style的参数需要参考echarts相关材料

https://lockdatav.blog.csdn.net/article/details/106379134





In [8]:
map_data=[[
  [120.09301091533, 30.237355077676],
  [120.09300231512, 30.237102585171],
  [120.09273277667, 30.237014001726],
  [120.09235667235, 30.237023849668],
  [120.09247034134, 30.237020799186],
  [120.09235415838, 30.236931281436],
  [120.09249523307, 30.237077424673],
  [120.09227857692, 30.236953488801],
  [120.09236889214, 30.236802403617],
  [120.09228327523, 30.236774501485],
  [120.0915479786, 30.237013030973],
  [120.09170832511, 30.236965232782],
  [120.09165932082, 30.236819989034],
  [120.09141982762, 30.236834325375],
  [120.09105978327, 30.237018774407],
  [120.09122130957, 30.236776979775],
  [120.09096738259, 30.23666355983],
  [120.09029968116, 30.236770616438],
  [120.08983820472, 30.237106950056],
  [120.08945597294, 30.23699652666],
  [120.0876611843, 30.237218331451],
  [120.08669674356, 30.236824529725],
  [120.08610557234, 30.237036276042],
  [120.08564989625, 30.237442457947],
  [120.08522299762, 30.237438959045],
  [120.08367382005, 30.23833648123],
  [120.0831192649, 30.238661839459],
  [120.0824703985, 30.239346940464]],
 [[120.11135706939, 30.242656396185],
  [120.11263983781, 30.243551891713],
  [120.11295976437, 30.244109103119],
  [120.1122553903, 30.245760561279],
  [120.11182005283, 30.245738369889],
  [120.11221032939, 30.245853865175],
  [120.11194906755, 30.247118699308],
  [120.11217509127, 30.247369622311],
  [120.11225937415, 30.247291474057],
  [120.11228505787, 30.24755526687],
  [120.11367153178, 30.249148432492]]]

In [9]:
from pyecharts.globals import BMapType, ChartType
#map_data = [[y["coord"] for y in x] for x in data]

(
    BMap(init_opts=opts.InitOpts(width="1600px", height="800px"))
    .add_schema(
        baidu_ak="K2n5m9GXasQFV5arkjpCGsXL",
        center=[120.13066322374, 30.240018034923],
        zoom=15,
        is_roam=True,
        map_style={
            "styleJson": [
                {
                    "featureType": "water",
                    "elementType": "all",
                    "stylers": {"color": "#d1d1d1"},
                },
                {
                    "featureType": "land",
                    "elementType": "all",
                    "stylers": {"color": "#f3f3f3"},
                },
                {
                    "featureType": "railway",
                    "elementType": "all",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "highway",
                    "elementType": "all",
                    "stylers": {"color": "#fdfdfd"},
                },
                {
                    "featureType": "highway",
                    "elementType": "labels",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "arterial",
                    "elementType": "geometry",
                    "stylers": {"color": "#fefefe"},
                },
                {
                    "featureType": "arterial",
                    "elementType": "geometry.fill",
                    "stylers": {"color": "#fefefe"},
                },
                {
                    "featureType": "poi",
                    "elementType": "all",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "green",
                    "elementType": "all",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "subway",
                    "elementType": "all",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "manmade",
                    "elementType": "all",
                    "stylers": {"color": "#d1d1d1"},
                },
                {
                    "featureType": "local",
                    "elementType": "all",
                    "stylers": {"color": "#d1d1d1"},
                },
                {
                    "featureType": "arterial",
                    "elementType": "labels",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "boundary",
                    "elementType": "all",
                    "stylers": {"color": "#fefefe"},
                },
                {
                    "featureType": "building",
                    "elementType": "all",
                    "stylers": {"color": "#d1d1d1"},
                },
                {
                    "featureType": "label",
                    "elementType": "labels.text.fill",
                    "stylers": {"color": "#999999"},
                },
            ]
        },
    )
    .add(
        series_name="",
        type_=ChartType.LINES,
        data_pair=map_data,
        is_polyline=True,
        is_large=True,
        linestyle_opts=opts.LineStyleOpts(color="purple", opacity=0.6, width=1),
        effect_opts=opts.EffectOpts(trail_length=0.5),
    )
    .add_control_panel(
        copyright_control_opts=opts.BMapCopyrightTypeOpts(position=3),
        maptype_control_opts=opts.BMapTypeControlOpts(
            type_=BMapType.MAPTYPE_CONTROL_DROPDOWN
        ),
        scale_control_opts=opts.BMapScaleControlOpts(),
        overview_map_opts=opts.BMapOverviewMapControlOpts(is_open=True),
        navigation_control_opts=opts.BMapNavigationControlOpts(),
        geo_location_control_opts=opts.BMapGeoLocationControlOpts(),
    )
    .render_notebook()
)

## 课间题

 * 通过bmap，画出你家乡城市的地图，通过自定义坐标点，在家乡城市地图上画出你的家，小学，中学的位置，和任何你喜欢的地方

In [30]:
data_pair=[('乐清中学', 10)]

In [31]:
BAIDU_AK = "K2n5m9GXasQFV5arkjpCGsXL"
b1=BMap()
b1.add_schema(
            baidu_ak=BAIDU_AK,
            center=[120.95280482031248, 28.076062773010392],zoom=8,map_style={"style":'hardedge'}
        )
b1.add(
            "bmap",
            data_pair,
            label_opts=opts.LabelOpts(formatter="{b}"),
        )
b1.set_global_opts(
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=2000,pos_bottom=50),
            title_opts=opts.TitleOpts(title="BMap-基本示例"),
        )
b1.render_notebook()